In [2]:
from sklearn.metrics import f1_score
import ast


data=[]
prediction_new = []
label_new = []

# Read the file and extract the prediction and label columns
line = "17625	['O', 'O', 'O', 'O', 'O', 'O', 'X', 'O', 'O', '[CLS]', 'O', 'O', 'O', 'X', 'O', 'O', 'O', 'O', 'O', 'O']	['B-A0', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-V', 'B-A2', 'I-A2', 'O', 'B-A1', 'I-A1', 'I-A1', 'O', 'O', 'O', 'O']"


uid, prediction, label = line.strip().split("\t")
data.append((prediction.strip("[]"), label.strip("[]")))

print(f"data: type: {type(data)}")
print(f"prediction: {prediction}")
print(f"label: {label}")
# convert list to string
predictions_data = [d[0] for d in data]
labels_data = [d[1] for d in data]

lineTok = ast.literal_eval(label)
lineLab = ast.literal_eval(prediction)

print(f"lineTok: {lineTok}")
print(f"lineLab: {lineLab}")
for (Tok, Lab) in zip(lineTok, lineLab):
    
    if Lab in ['[CLS]','[SEP]', 'X']: # replace non-text tokens with O. These will not be evaluated.
        prediction_new.append('O')
        label_new.append('O')
        continue
    if(Lab == "B-V"):
        prediction_new.append("V")
    else:
        prediction_new.append(Lab)
        label_new.append(Tok)
        
print(f"prediction_new: {prediction_new}")
print(f"label_new: {label_new}")

from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import recall_score as class_recall_score

new1 = [label_new]
print(f"new1: {new1}")
new2 = [prediction_new]

result_f1 = f1_score(label_new, prediction_new,average="micro")

print(f"result_f1: {result_f1}")

data: type: <class 'list'>
prediction: ['O', 'O', 'O', 'O', 'O', 'O', 'X', 'O', 'O', '[CLS]', 'O', 'O', 'O', 'X', 'O', 'O', 'O', 'O', 'O', 'O']
label: ['B-A0', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-V', 'B-A2', 'I-A2', 'O', 'B-A1', 'I-A1', 'I-A1', 'O', 'O', 'O', 'O']
lineTok: ['B-A0', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-V', 'B-A2', 'I-A2', 'O', 'B-A1', 'I-A1', 'I-A1', 'O', 'O', 'O', 'O']
lineLab: ['O', 'O', 'O', 'O', 'O', 'O', 'X', 'O', 'O', '[CLS]', 'O', 'O', 'O', 'X', 'O', 'O', 'O', 'O', 'O', 'O']
prediction_new: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
label_new: ['B-A0', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-A2', 'I-A2', 'O', 'O', 'I-A1', 'I-A1', 'O', 'O', 'O', 'O']
new1: [['B-A0', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-A2', 'I-A2', 'O', 'O', 'I-A1', 'I-A1', 'O', 'O', 'O', 'O']]
result_f1: 0.75


In [60]:
data = []
labels = []
predictions = []

filePath = "./output/conllsrl_test_predictions_29.tsv"
with open(filePath, "r") as f:
    for line in f:
        # delete the first line
        if "uid" in line:
            continue
        uid, prediction, label = line.strip().split("\t")
        labels.append(label)
        predictions.append(prediction)
        data.append((prediction.strip("[]"), label.strip("[]")))

print("prediction: ", type(data))

prediction:  <class 'list'>


In [3]:
data = []
labels = []
predictions = []

filePath = "./output/conllsrl_test_predictions_29.tsv"
with open(filePath, "r") as f:
    for line in f:
        # delete the first line
        if "uid" in line:
            continue
        uid, prediction, label = line.strip().split("\t")
        labels.append(label)
        predictions.append(prediction)
        data.append((prediction.strip("[]"), label.strip("[]")))

print("prediction: ", type(data))
# apply this conversion to all the elements in the list
lineToks = [ast.literal_eval(label) for label in labels]
lineLabs = [ast.literal_eval(prediction) for prediction in predictions]

trueLabels = [] # labels
predictLabels = [] # predictions

for lineTok, lineLab in zip(lineToks, lineLabs):
    if lineLab in ['[CLS]','[SEP]', 'X']: # replace non-text tokens with O. These will not be evaluated.
        predictLabels.append('O')
        trueLabels.append('O')
        continue
    if(lineLab == "B-V"):
        predictLabels.append("V")
    else:
        predictLabels.append(lineLab)
        trueLabels.append(lineTok) 
 
# flatten the list of lists into a single list
trueLabels = [item for sublist in trueLabels for item in sublist]
predictLabels = [item for sublist in predictLabels for item in sublist]
       
result_f1 = f1_score(trueLabels, predictLabels,average="micro")

print(f"result_f1: {result_f1}")

prediction:  <class 'list'>
result_f1: 0.761921602045164


----


In [22]:
import torch

from transformers import BertTokenizer, BertModel, BertConfig

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt

# Load pre-trained model tokenizer (vocabulary) dmis-lab/biobert-base-cased-v1.2
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True,truncation=True)

tokenizer = BertTokenizer.from_pretrained('dmis-lab/biobert-base-cased-v1.2', do_lower_case=True,truncation=True)



In [23]:
#text = "the a/t-rich mut sequence indicates that normal splicing was abolished by a g-to-a transition at the first nucleotide of intron 2."
text2 = "disruption of rpos, mlra or csga, and the curli subunit gene being able to abolish ha and curli production by strain chi7122 interrupt this open reading frame."
text1 = "torc induction was abolished by deletion of the distal tor box (box1), which interrupt this open reading frame."

text = "in addition, deletion of the distal tor box (box1) abolished torc induction whereas the presence of a dna fragment starting three bases upstream from box1 suffices for normal torc expression."


marked_text = "[CLS] " + text + " [SEP]"

# Tokenize our sentence with the BERT tokenizer.
tokenized_text = tokenizer.tokenize(marked_text)
tokenized_text2 = tokenizer.tokenize("[CLS] " + text2 + " [SEP]")
# Print out the tokens.
print (len(tokenized_text))
print (tokenized_text2)

47
['[CLS]', 'disruption', 'of', 'r', '##po', '##s', ',', 'm', '##l', '##ra', 'or', 'c', '##s', '##ga', ',', 'and', 'the', 'curl', '##i', 'subunit', 'gene', 'being', 'able', 'to', 'a', '##bol', '##ish', 'ha', 'and', 'curl', '##i', 'production', 'by', 'strain', 'ch', '##i', '##7', '##12', '##2', 'interrupt', 'this', 'open', 'reading', 'frame', '.', '[SEP]']


In [24]:
# Map the token strings to their vocabulary indeces.
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
indexed_tokens2 = tokenizer.convert_tokens_to_ids(tokenized_text2)
# # Display the words with their indeces.
# for tup in zip(tokenized_text, indexed_tokens):
#     print('{:<12} {:>6,}'.format(tup[0], tup[1]))

In [25]:
out = tokenizer.encode_plus(text = indexed_tokens, add_special_tokens=False,
                                        truncation_strategy ='only_first',
                                        max_length = 50, pad_to_max_length=True)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/phatpham/anaconda3/envs/min_ds-env/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [26]:
tokenIds = out['input_ids']
typeIds = out['token_type_ids']

print(f"tokenIds: {tokenIds}")

tokenIds: [101, 1107, 1901, 117, 3687, 26883, 1320, 1104, 1103, 4267, 19760, 1106, 1197, 2884, 113, 2884, 1475, 114, 8632, 1106, 19878, 18293, 6142, 1103, 2915, 1104, 170, 173, 1605, 17906, 2547, 1210, 7616, 15011, 1121, 2884, 1475, 28117, 3101, 18117, 1111, 2999, 1106, 19878, 2838, 119, 102, 0, 0, 0]


In [27]:
# Mark each of the 29 tokens as belonging to sentence "1".
segments_ids = [1] * len(tokenized_text)
segments_ids2 = [1] * len(tokenized_text2)
print (segments_ids)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
#making and loading model
import torch
from models.model import multiTaskModel
from utils.data_utils import TaskType, ModelType, NLP_MODELS

# Load pre-trained model (weights)
loadedDict = torch.load('./output/multi_task_model_9_13050.pt', map_location=torch.device('cpu'))

taskParams = loadedDict['task_params']

modelName =taskParams.modelType.name.lower()
_, _ , tokenizerClass, defaultName = NLP_MODELS[modelName]
configName = taskParams.modelConfig

#making tokenizer for model
tokenizer = tokenizerClass.from_pretrained(configName)

allParams = {}
allParams['task_params'] = taskParams
allParams['gpu'] = torch.cuda.is_available()
# dummy values
allParams['num_train_steps'] = 10
allParams['warmup_steps'] = 0
allParams['learning_rate'] = 2e-5
allParams['epsilon'] = 1e-8

model = multiTaskModel(allParams)


In [ ]:
attension_masks = torch.tensor([[1] * len(tokenized_text)])
output = model.network(tokens_tensor, segments_tensors, attension_masks, 0, 'conllsrl')
output[1]

In [ ]:
token_embeddings = torch.stack(hidden_states, dim=0)
token_embeddings = torch.squeeze(token_embeddings, dim=1)
token_embeddings = token_embeddings.permute(1,0,2)

token_embeddings.size()

# Stores the token vectors, with shape [22 x 3,072]
token_vecs_cat = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:
    
    # `token` is a [12 x 768] tensor

    # Concatenate the vectors (that is, append them together) from the last 
    # four layers.
    # Each layer vector is 768 values, so `cat_vec` is length 3,072.
    cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)
    
    # Use `cat_vec` to represent `token`.
    token_vecs_cat.append(cat_vec)

print ('Shape is: %d x %d' % (len(token_vecs_cat), len(token_vecs_cat[0])))
print(token_vecs_cat)

In [ ]:
text = "in addition, deletion of the distal tor box (box1) abolished torc induction whereas the presence of a dna fragment starting three bases upstream from box1 suffices for normal torc expression."
import sys
sys.path.insert(1, '../')
from infer_pipeline import inferPipeline
from utils.data_utils import TaskType
pipe = inferPipeline(modelPath='./output/multi_task_model_8_367.pt', maxSeqLen=50)



In [ ]:
#cosine similarity between two word

In [ ]:
import torch
import torch.nn.functional as F

def cosine_sim(tensor1, tensor2):
    """
    Calculate the cosine similarity between two 2D tensors using PyTorch.

    Args:
        tensor1 (torch.Tensor): The first 2D tensor.
        tensor2 (torch.Tensor): The second 2D tensor.

    Returns:
        torch.Tensor: The cosine similarity between the two tensors.
    """
    # Flatten both tensors to 1D arrays
    flat_tensor1 = tensor1.view(-1)
    flat_tensor2 = tensor2.view(-1)

    # Compute the dot product
    dot_product = torch.dot(flat_tensor1, flat_tensor2)

    # Compute the L2 (Euclidean) norms
    norm_tensor1 = torch.norm(flat_tensor1)
    norm_tensor2 = torch.norm(flat_tensor2)

    # Calculate the cosine similarity
    similarity = dot_product / (norm_tensor1 * norm_tensor2)

    return similarity

def cosine_similarity(tensor1, tensor2):
    # Pad tensor2 to match the size of tensor1
    height_diff = tensor1.shape[0] - tensor2.shape[0]
    width_diff = tensor1.shape[1] - tensor2.shape[1]

    padded_tensor2 = F.pad(tensor2, (0, width_diff, 0, height_diff))

    # Now, both tensors have the same size, and you can calculate cosine similarity as before
    similarity = cosine_sim(tensor1, padded_tensor2)

    return similarity
# Example usage

similarity = cosine_similarity(token_vecs, token_vecs2)
print("Cosine Similarity:", similarity.item())  # Convert to a Python float



In [ ]:
print("Cosine module similarity:", consine_module_similarity(token_vecs, token_vecs2))

In [ ]:
# Concatenate the tensors for all layers. We use `stack` here to
# create a new dimension in the tensor.
token_embeddings = torch.stack(hidden_states, dim=0)

token_embeddings.size()

In [ ]:
# Remove dimension 1, the "batches".
token_embeddings = torch.squeeze(token_embeddings, dim=1)

token_embeddings.size()

In [ ]:
    
# Concatenate the tensors for all layers. We use `stack` here to
# create a new dimension in the tensor. 
token_embeddings = torch.stack(hidden_states, dim=0) # torch.Size([13, 1, 50, 768])

# Remove dimension 1, the "batches".
token_embeddings = torch.squeeze(token_embeddings, dim=1) # torch.Size([13, 50, 768])

# Swap dimensions 0 and 1.
token_embeddings = token_embeddings.permute(1,0,2) # torch.Size([50, 13, 768])

    
    # ## CONCATENATE the last four layers
    # # Stores the token vectors, with shape [22 x 3,072]
    # token_vecs_cat = []

    # # `token_embeddings` is a [22 x 12 x 768] tensor.

    # # For each token in the sentence...
    # for token in token_embeddings:
        
    #     # `token` is a [12 x 768] tensor

    #     # Concatenate the vectors (that is, append them together) from the last 
    #     # four layers.
    #     # Each layer vector is 768 values, so `cat_vec` is length 3,072.
    #     cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)
        
    #     # Use `cat_vec` to represent `token`.
    #     token_vecs_cat.append(cat_vec)  # Shape is: 50 x 3072

    # print ('Shape is: %d x %d' % (len(token_vecs_cat), len(token_vecs_cat[0]))) 
    
    # ## SUMMING LAST FOUR LAYERS
    # # Stores the token vectors, with shape [22 x 768]
    # token_vecs_sum = []

    # # `token_embeddings` is a [22 x 12 x 768] tensor.

    # # For each token in the sentence...
    # for token in token_embeddings:

    #     # `token` is a [50 x 768] tensor

    #     # Sum the vectors from the last four layers.
    #     sum_vec = torch.sum(token[-4:], dim=0)
        
    #     # Use `sum_vec` to represent `token`.
    #     token_vecs_sum.append(sum_vec)

    # print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0]))) # Shape is: 50 x 768
    

In [ ]:
data = read_data('../data/coNLL_tsv/bert-base-uncased_prepared_data/ner_conll_testa_abolish.json')
    
print("data shape: ", len(data))  # 280
tokens_id = data[0]['token_id']
segments_id = data[0]['type_id']


# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([tokens_id])
segments_tensors = torch.tensor([segments_id])

# Load pre-trained model (weights)
model = BertModel.from_pretrained('dmis-lab/biobert-base-cased-v1.2',
                                output_hidden_states = True # Whether the model returns all hidden-states.
                                )
# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

with torch.no_grad():

    outputs = model(tokens_tensor, segments_tensors)
    print("outputs: ", len(outputs)) # 2
    hidden_states = outputs[2]
    
    
print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")  #13
layer_i = 0

print ("Number of batches:", len(hidden_states[layer_i])) #1
batch_i = 0

print ("Number of tokens:", len(hidden_states[layer_i][batch_i])) #50
token_i = 0

print ("Number of hidden units:", len(hidden_states[layer_i][batch_i][token_i]))     # 768


# `hidden_states` is a Python list.
print('      Type of hidden_states: ', type(hidden_states)) # <class 'tuple'>

# Each layer in the list is a torch tensor.
print('Tensor shape for each layer: ', hidden_states[0].size()) # torch.Size([1, 50, 768])


## Sentence Vectors
# `hidden_states` has shape [13 x 1 x 50 x 768]

# `token_vecs` is a tensor with shape [50 x 768]
token_vecs = hidden_states[-2][0]

# Calculate the average of all 22 token vectors.
sentence_embedding = torch.mean(token_vecs, dim=0)
print ("Our final sentence embedding vector of shape:", sentence_embedding.size()) # torch.Size([768])

In [17]:
import pickle
with open('models/outputs.pkl', 'rb') as f:
    token_vecs_cat = pickle.load(f)

print("token_vecs_cat: ", len(token_vecs_cat)) # 50
print(token_vecs_cat['uid']) # (3072,)


token_vecs_cat:  2
['22110', '22111', '22112', '22113', '22114', '22115', '22116', '22117', '22118', '22119', '22120', '22121', '22122', '22123', '22124', '22125', '22126', '22127', '22128', '22129', '22130', '22131', '22132', '22133', '22134', '22135', '22136', '22137', '22138', '22139', '22140', '22141']
